# TP 01 - Le Mécanisme d'Attention

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Comprendre et implémenter le mécanisme d'attention, brique fondamentale des Transformers

---

## Objectifs pédagogiques

À la fin de ce TP, vous serez capable de :
1. Expliquer intuitivement ce qu'est l'attention
2. Comprendre les concepts de Query, Key, Value
3. Implémenter le Scaled Dot-Product Attention
4. Visualiser et interpréter les poids d'attention

## 0. Installation et imports

Exécutez cette cellule pour installer les dépendances nécessaires.

In [ ]:
# Installation des dépendances (Google Colab)
!pip install torch matplotlib numpy -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# Configuration
torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")
print(f"GPU disponible: {torch.cuda.is_available()}")

---

## 1. Introduction : Pourquoi l'attention ?

> **Note pédagogique** : Dans les sessions 1 à 3, on se concentre sur le **fonctionnement** de l'architecture (inférence/forward pass). L'**entraînement** (backpropagation, optimisation) sera abordé en session 4.

### 1.1 Les architectures séquentielles (RNN / LSTM)

Les **réseaux récurrents (RNN)** traitent les séquences **mot par mot** :

```
        ┌───┐    ┌───┐    ┌───┐    ┌───┐    ┌───┐
  x₁ ──▶│ h ├───▶│ h ├───▶│ h ├───▶│ h ├───▶│ h ├──▶ sortie
  Le    └───┘    └───┘    └───┘    └───┘    └───┘
           │        │        │        │        │
          x₂       x₃       x₄       x₅       x₆
         chat     dort      sur      le     canapé
```

**Problème** : L'information passe de cellule en cellule. Pour relier "Le chat" à "canapé", il faut traverser toute la chaîne → l'info se dégrade (gradient évanescent).

Les **LSTM** ajoutent des "portes" pour mieux contrôler la mémoire :

```
                    ┌─────────────────────────┐
                    │      CELLULE LSTM       │
        ┌───────────┬───────────┬─────────────┤
        │  Porte    │  Porte    │    Porte    │
        │  Oubli    │  Entrée   │   Sortie    │
        └───────────┴───────────┴─────────────┘
              │           │            │
          Effacer?    Ajouter?    Utiliser?
```

**Amélioration** : Les LSTM retiennent mieux les infos longue distance.
**Mais** : Toujours séquentiel (lent) et limité sur les très longues séquences.

### 1.2 L'architecture Transformer

Le **Transformer** (2017) abandonne la récurrence. Chaque mot peut regarder **tous les autres directement** :

```
Entrée: "Le chat dort sur le canapé" (6 tokens)
         │    │    │    │    │    │
         ▼    ▼    ▼    ▼    ▼    ▼
┌─────────────────────────────────────────────┐
│          EMBEDDINGS (6 vecteurs)            │
└─────────────────────────────────────────────┘
         │    │    │    │    │    │
         ▼    ▼    ▼    ▼    ▼    ▼
┌─────────────────────────────────────────────┐
│             SELF-ATTENTION                  │
│   Chaque vecteur regarde les 5 autres       │
│   → Enrichit chaque mot avec le CONTEXTE    │
└─────────────────────────────────────────────┘
         │    │    │    │    │    │
         ▼    ▼    ▼    ▼    ▼    ▼
       (6 vecteurs enrichis)
         │    │    │    │    │    │
         ▼    ▼    ▼    ▼    ▼    ▼
┌─────────────────────────────────────────────┐
│        FEED-FORWARD (par position)          │
│   Exploite le contexte enrichi              │
│   (comme un réseau de neurones classique)   │
└─────────────────────────────────────────────┘
         │    │    │    │    │    │
         ▼    ▼    ▼    ▼    ▼    ▼
      Sortie: 6 vecteurs transformés
```

**Points clés** :
- **Entrée = Sortie** : Si tu entres 6 mots → tu obtiens 6 vecteurs enrichis
- **Taille variable** : Tu peux entrer 5, 50, ou 500 mots (jusqu'à une limite : 512 pour BERT, 128K pour GPT-4)
- **Self-Attention** : Donne du contexte à chaque mot
- **Feed-Forward** : Exploite ce contexte (transformation non-linéaire)

**Que sort le Transformer ?**

Le Transformer produit des **vecteurs enrichis** (représentations). Une couche de sortie (ajoutée selon la tâche) les transforme en résultat :
- **Classification** → probabilité par classe (ex: 70% positif, 30% négatif)
- **Génération** → probabilité du prochain mot
- **Traduction** → phrase dans l'autre langue

### Comment les mots entrent dans le Transformer ?

Chaque mot passe par **deux étapes** avant d'entrer :

```
Mot "chat" (position 1)
        │
        ▼
┌─────────────────────────────────────────────────┐
│ Token Embedding (fixe pour chaque token)        │
│ "chat" → [0.8, 0.1, 0.3, ...]                   │
└─────────────────────────────────────────────────┘
        │
        + (addition)
        │
┌─────────────────────────────────────────────────┐
│ Positional Encoding (fixe pour chaque position) │
│ position 1 → [0.0, 0.1, 0.0, ...]               │
└─────────────────────────────────────────────────┘
        │
        ▼
Vecteur d'entrée = [0.8, 0.2, 0.3, ...]
```

**Deux composants distincts, tous deux figés après entraînement :**

| Composant | Taille | Rôle |
|-----------|--------|------|
| Token embeddings | ~50k × dim | "Qui suis-je ?" (sens du mot) |
| Positional encodings | max_len × dim | "Où suis-je ?" (position dans la phrase) |

**Pourquoi c'est important ?** Sans le positional encoding, le modèle ne distinguerait pas :
- *"Le chat mange la souris"*
- *"La souris mange le chat"*

(Mêmes tokens, ordre différent → sens opposé !)

### 1.3 Empilement des blocs

Ces blocs (Attention + FFN) sont **empilés** : la sortie de l'un devient l'entrée du suivant.

```
Entrée (6 vecteurs)
         │
         ▼
┌─────────────────┐
│   Attention 1   │
│        ↓        │  Bloc 1
│     FFN 1       │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│   Attention 2   │
│        ↓        │  Bloc 2
│     FFN 2       │
└────────┬────────┘
         │
        ...
         │
         ▼
┌─────────────────┐
│   Attention N   │
│        ↓        │  Bloc N (ex: N=12 pour BERT)
│     FFN N       │
└────────┬────────┘
         │
         ▼
Sortie (6 vecteurs très enrichis)
```

Chaque passage enrichit les représentations. Après N blocs, chaque mot "comprend" toute la phrase.

### 1.4 Ce qu'on va construire

```
    TRANSFORMER
    ┌────────────────────────────┐
    │  Embedding + Positional    │
    ├────────────────────────────┤
    │ ┌────────────────────────┐ │
    │ │   SELF-ATTENTION  ◀────┼─┼─── Sessions 1-2
    │ └────────────────────────┘ │
    │ ┌────────────────────────┐ │
    │ │     FEED-FORWARD       │ │
    │ └────────────────────────┘ │
    │         × N blocs          │
    ├────────────────────────────┤
    │     Couche de sortie       │
    └────────────────────────────┘
```

**Plan du cours** :
- **Session 1** : Mécanisme d'attention (ce TP)
- **Session 2** : Multi-Head Attention
- **Session 3** : Assembler le Transformer complet
- **Sessions 4-5** : Entraînement et projets

### 1.5 L'idée clé de l'attention

L'attention répond à la question : **"Pour comprendre ce mot, quels autres mots dois-je regarder ?"**

**Exemple** : *"Le chat qui dormait sur le canapé a sauté"*
- Pour comprendre **"a sauté"** → regarder **"chat"** (le sujet, pas "canapé")

### Analogie : La bibliothèque

- **Query (Q)** : Votre question ("Je cherche un livre sur les chats")
- **Key (K)** : Les mots-clés de chaque livre
- **Value (V)** : Le contenu des livres

L'attention compare votre **question** aux **mots-clés**, puis retourne un mélange pondéré des **contenus** les plus pertinents.

---

### 📚 Pour approfondir RNN/LSTM (optionnel)

**Vidéos en français** :
- [Machine Learnia - Les RNN expliqués](https://www.youtube.com/watch?v=EL439RMv3Xc) (~20 min)
- [Science4All - Comprendre les LSTM](https://www.youtube.com/watch?v=WCUNPb-5EYI) (~15 min)

**Articles en français** :
- [Pensée Artificielle - Introduction aux RNN](https://www.penseeartificielle.fr/comprendre-reseaux-neurones-recurrents-rnn/)
- [DataScientest - LSTM expliqué simplement](https://datascientest.com/lstm-tout-savoir)

---

## 2. Visualisation intuitive

Avant de coder, visualisons ce que fait l'attention.

In [ ]:
# Exemple simple : attention dans une phrase
phrase = ["Le", "chat", "mange", "la", "souris"]

# Matrice d'attention simulée (quels mots regardent quels mots ?)
# Chaque ligne = un mot qui "regarde" les autres
attention_simulee = torch.tensor([
    [0.8, 0.1, 0.05, 0.03, 0.02],  # "Le" regarde surtout lui-même
    [0.1, 0.7, 0.1, 0.05, 0.05],   # "chat" regarde surtout lui-même
    [0.05, 0.4, 0.4, 0.05, 0.1],   # "mange" regarde "chat" et lui-même
    [0.02, 0.03, 0.05, 0.8, 0.1],  # "la" regarde surtout lui-même
    [0.02, 0.1, 0.2, 0.08, 0.6],   # "souris" regarde "mange" et elle-même
])

# Visualisation
plt.figure(figsize=(8, 6))
plt.imshow(attention_simulee, cmap='Blues')
plt.xticks(range(5), phrase)
plt.yticks(range(5), phrase)
plt.xlabel("Mots regardés (Keys)")
plt.ylabel("Mots qui regardent (Queries)")
plt.title("Qui regarde qui ? (Matrice d'attention)")
plt.colorbar(label="Poids d'attention")

# Afficher les valeurs
for i in range(5):
    for j in range(5):
        plt.text(j, i, f'{attention_simulee[i,j]:.2f}', 
                ha='center', va='center',
                color='white' if attention_simulee[i,j] > 0.5 else 'black')
plt.show()

**Question** : Dans cette matrice, quel mot le verbe "mange" regarde-t-il le plus ? Pourquoi est-ce logique ?

---

## 3. Scaled Dot-Product Attention

Le **Scaled Dot-Product Attention** est l'opération qui calcule la **matrice d'attention** (les poids "qui regarde qui") et produit les vecteurs enrichis en sortie.

**Rappel des 3 vecteurs :**

| Vecteur | Rôle | Sert à... |
|---------|------|-----------|
| **Q** (Query) | Ce que je cherche | Calculer les poids (avec K) |
| **K** (Key) | Mon identité / étiquette | Calculer les poids (avec Q) |
| **V** (Value) | Mon contenu / l'info que je transmets | Être récupéré selon les poids |

**Concrètement** : La matrice d'attention dit "à quel point chaque mot m'intéresse" (calculée avec Q et K). Ensuite on récupère l'**information** (V) de ces mots, pondérée par ces poids.

**Exemple** : Pour "dort" dans ["Le", "chat", "dort"], si les poids sont [0.26, 0.42, 0.32] :
- On récupère 26% du **contenu** (V) de "Le"
- On récupère 42% du **contenu** (V) de "chat"
- On récupère 32% du **contenu** (V) de "dort"

**Attention au vocabulaire** :
- `softmax(QK^T/√d_k)` = **matrice d'attention** (les poids)
- `Attention(Q,K,V)` = matrice d'attention × V = **sortie** (vecteurs enrichis)

### La formule

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Où :
- $Q$ (Query) : Ce que je cherche - shape `(seq_len, d_k)`
- $K$ (Key) : Les étiquettes de ce qui est disponible - shape `(seq_len, d_k)`
- $V$ (Value) : Le contenu disponible - shape `(seq_len, d_v)`
- $d_k$ : Dimension des clés (pour normaliser)

> **Note** : Q, K, V sont obtenus à partir des embeddings via des matrices de poids apprenables. Cela permet à chaque mot d'avoir une représentation adaptée à son rôle (chercher, s'identifier, transmettre).

### Exemple concret

Prenons la phrase **["Le", "chat", "dort"]** avec des embeddings de dimension 4.

Supposons qu'après transformation, on obtienne :

```
         Q (Queries)         K (Keys)           V (Values)
Le    → [0.1, 0.2, 0.1, 0.0]  [0.9, 0.1, 0.0, 0.2]  [1.0, 0.0, 0.0, 0.0]
chat  → [0.2, 0.8, 0.1, 0.3]  [0.2, 0.9, 0.2, 0.1]  [0.0, 1.0, 0.0, 0.0]
dort  → [0.3, 0.7, 0.2, 0.1]  [0.1, 0.3, 0.8, 0.1]  [0.0, 0.0, 1.0, 0.0]
```

**Calculons l'attention pour "dort"** (quelle info récupère-t-il des autres mots ?) :

**Étape 1 - Scores (Q·Kᵀ)** : On compare la Query de "dort" aux Keys de tous les mots
```
Q_dort · K_Le   = 0.3×0.9 + 0.7×0.1 + 0.2×0.0 + 0.1×0.2 = 0.36
Q_dort · K_chat = 0.3×0.2 + 0.7×0.9 + 0.2×0.2 + 0.1×0.1 = 0.74  ← score élevé !
Q_dort · K_dort = 0.3×0.1 + 0.7×0.3 + 0.2×0.8 + 0.1×0.1 = 0.41

Scores = [0.36, 0.74, 0.41]
```

**Étape 2 - Scaling (÷√d_k)** : On divise par √4 = 2
```
Scaled = [0.18, 0.37, 0.205]
```

**Étape 3 - Softmax** : On transforme en probabilités
```
Poids = [0.26, 0.42, 0.32]  (somme = 1)
```

**Étape 4 - Output (poids × V)** : Moyenne pondérée des Values
```
Output_dort = 0.26 × V_Le + 0.42 × V_chat + 0.32 × V_dort
            = 0.26 × [1,0,0,0] + 0.42 × [0,1,0,0] + 0.32 × [0,0,1,0]
            = [0.26, 0.42, 0.32, 0.0]
```

**Interprétation** : La nouvelle représentation de "dort" contient **42% d'info de "chat"** (le sujet), **32% de lui-même** (le verbe), et **26% de "Le"** (le déterminant). Le modèle a appris que pour comprendre un verbe, il faut surtout regarder son sujet.

### Décomposition étape par étape

1. **Scores** : $QK^T$ - Mesure la similarité entre queries et keys
2. **Scaling** : Division par $\sqrt{d_k}$ - Évite des valeurs trop grandes
3. **Softmax** : Transforme en probabilités (somme = 1)
4. **Output** : Multiplication par $V$ - Moyenne pondérée des values

### Pourquoi softmax ? Pourquoi normaliser ?

**Le softmax** transforme des scores quelconques en **probabilités** :

$$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$$

```
Scores bruts :  [0.36, 0.74, 0.41]  (peuvent être négatifs, grands, etc.)
                        ↓ softmax
Probabilités :  [0.26, 0.42, 0.32]  (entre 0 et 1, somme = 1)
```

**Propriétés utiles** :
- Toutes les valeurs sont positives et somment à 1 → interprétables comme "pourcentage d'attention"
- Amplifie les différences : le score le plus élevé "gagne" plus de poids

**La normalisation (÷√d_k)** évite un problème quand la dimension est grande :

```
Sans normalisation (d_k = 512) :
  Scores Q·K → valeurs entre -50 et +50
  Softmax → [0.0001, 0.9998, 0.0001]  ← trop "peaked" !
  
Avec normalisation (÷√512 ≈ 22.6) :
  Scores → valeurs entre -2 et +2
  Softmax → [0.20, 0.45, 0.35]  ← distribution plus douce
```

Une distribution trop "peaked" pose problème : gradients très faibles → apprentissage difficile.

### Exercice 1 : Calcul manuel des scores

Commençons par calculer les scores d'attention manuellement.

In [ ]:
# Exemple simple avec 3 mots et dimension 4
seq_len = 3
d_k = 4

# Créons des Query, Key, Value aléatoires
Q = torch.randn(seq_len, d_k)
K = torch.randn(seq_len, d_k)
V = torch.randn(seq_len, d_k)

print("Q (Queries):")
print(Q)
print(f"\nShape Q: {Q.shape}")
print(f"Shape K: {K.shape}")
print(f"Shape V: {V.shape}")

In [ ]:
# ============================================
# EXERCICE 1 : Calculez les scores d'attention
# ============================================

# Étape 1 : Calculer QK^T (produit matriciel)
# La transposée de K se note K.T

scores = None  # TODO: Calculer QK^T

print("Scores (QK^T):")
print(scores)
print(f"Shape: {scores.shape}")  # Devrait être (3, 3)

In [ ]:
# ============================================
# EXERCICE 2 : Appliquez le scaling
# ============================================

# Diviser par la racine de la dimension des vecteurs pour éviter des valeurs trop grandes

import math

scaled_scores = None  # TODO: scores / sqrt(d_k)

print("Scaled scores:")
print(scaled_scores)

In [ ]:
# ============================================
# EXERCICE 3 : Appliquez le softmax
# ============================================

# Le softmax transforme les scores en probabilités
# Chaque ligne doit sommer à 1
# Indice : F.softmax(tensor, dim=i) applique softmax sur la dimension i

attention_weights = None  # TODO: Appliquer softmax sur scaled_scores

print("Poids d'attention (après softmax):")
print(attention_weights)
print(f"\nVérification - Somme par ligne: {attention_weights.sum(dim=1)}")

In [ ]:
# ============================================
# EXERCICE 4 : Calculez la sortie finale
# ============================================

# Multiplier les poids d'attention par V
# C'est une moyenne pondérée des values

output = None  # TODO: attention_weights @ V

print("Output:")
print(output)
print(f"Shape: {output.shape}")  # Devrait être (3, 4)

---

## 4. Implémentation complète

### Exercice 5 : Fonction d'attention

Maintenant, regroupez tout dans une fonction.

In [ ]:
def scaled_dot_product_attention(Q, K, V):
    """
    Calcule le Scaled Dot-Product Attention.
    
    Args:
        Q: Queries, shape (seq_len, d_k) ou (batch, seq_len, d_k)
        K: Keys, shape (seq_len, d_k) ou (batch, seq_len, d_k)
        V: Values, shape (seq_len, d_v) ou (batch, seq_len, d_v)
    
    Returns:
        output: Résultat de l'attention, shape (seq_len, d_v)
        attention_weights: Poids d'attention, shape (seq_len, seq_len)
    """
    # TODO: Récupérer d_k (dernière dimension de K)
    d_k=None
    # TODO: Implémenter les 4 étapes
    # 1. Calculer les scores : QK^T
    scores = None
    
    # 2. Scaling : diviser par sqrt(d_k)
    scaled_scores = None
    
    # 3. Softmax pour obtenir les poids
    attention_weights = None
    
    # 4. Moyenne pondérée : weights @ V
    output = None
    
    return output, attention_weights

In [ ]:
# Test de votre fonction
Q_test = torch.randn(4, 8)  # 4 tokens, dimension 8
K_test = torch.randn(4, 8)
V_test = torch.randn(4, 8)

output, weights = scaled_dot_product_attention(Q_test, K_test, V_test)

print(f"Structure de sortie: {output.shape}")  # Attendu: (4, 8)
print(f"Structure des poids: {weights.shape}")  # Attendu: (4, 4)
print(f"Somme des poids par ligne: {weights.sum(dim=1)}")  # Attendu: [1, 1, 1, 1]

---

## 5. Pourquoi diviser par sqrt(d_k) ?

C'est une question importante ! Voyons l'effet du scaling.

In [ ]:
# Comparaison avec et sans scaling
d_k_grand = 512  # Dimension typique dans un Transformer

Q_grand = torch.randn(10, d_k_grand)
K_grand = torch.randn(10, d_k_grand)

# Scores sans scaling
scores_sans_scaling = Q_grand @ K_grand.T
attention_sans_scaling = F.softmax(scores_sans_scaling, dim=-1)

# Scores avec scaling
scores_avec_scaling = (Q_grand @ K_grand.T) / math.sqrt(d_k_grand)
attention_avec_scaling = F.softmax(scores_avec_scaling, dim=-1)

 # Fonction pour calculer l'entropie (avec epsilon pour éviter log(0))
def entropy(p, eps=1e-9):
  p_safe = p.clamp(min=eps)
  return -(p * p_safe.log()).sum(dim=-1).mean()


print("=== SANS SCALING ===")
print(f"Scores - min: {scores_sans_scaling.min():.2f}, max: {scores_sans_scaling.max():.2f}")
print(f"Attention max par ligne: {attention_sans_scaling.max(dim=-1).values}")
print(f"Entropie moyenne: {entropy(attention_sans_scaling):.4f}")

print("\n=== AVEC SCALING ===")
print(f"Scores - min: {scores_avec_scaling.min():.2f}, max: {scores_avec_scaling.max():.2f}")
print(f"Attention max par ligne: {attention_avec_scaling.max(dim=-1).values}")
print(f"Entropie moyenne: {entropy(attention_avec_scaling):.4f}")

**Observation** : Sans scaling, le softmax devient très "peaked" (une valeur proche de 1, les autres proches de 0). Le scaling permet une distribution plus douce et des gradients plus stables.

**Comment lire l'entropie ?**
- **Entropie haute** (~2.3 pour 10 tokens) → attention répartie sur plusieurs mots
- **Entropie basse** (~0) → attention concentrée sur un seul mot

**Nuance importante** : Une attention concentrée n'est pas toujours mauvaise ! Par exemple, dans *"Le chat dort, il ronfle"*, le mot "il" DOIT regarder "chat" à 95%.

Le problème c'est quand l'attention est peaked **par défaut** (artefact numérique du softmax saturé) plutôt que **par apprentissage**. Le scaling permet au modèle de **choisir** entre attention concentrée ou distribuée selon ce qui est pertinent.

---

## 6. Module nn.Module : Self-Attention

### Self-Attention vs Cross-Attention

Jusqu'ici, on a manipulé Q, K, V comme des tenseurs indépendants. Mais d'où viennent-ils ?

**Self-Attention** (ce qu'on fait ici) :
- Q, K, V sont tous calculés à partir du **même** input `x`
- Chaque mot de la phrase regarde les autres mots **de la même phrase**
- C'est le cas dans BERT, GPT, et la plupart des Transformers

```
x (embeddings) ──┬──► W_q ──► Q
                 ├──► W_k ──► K    (même source x)
                 └──► W_v ──► V
```

**Cross-Attention** (on verra plus tard) :
- Q vient d'une source, K et V d'une **autre** source
- Exemple : en traduction, le décodeur (français) "interroge" l'encodeur (anglais)
- Utilisé dans les architectures encodeur-décodeur

```
x_decoder ──► W_q ──► Q
x_encoder ──┬──► W_k ──► K    (sources différentes)
            └──► W_v ──► V
```

> **Dans ce TP**, on implémente la **self-attention** : la séquence "s'attentionne elle-même".

### Exercice 6 : Classe SelfAttention en PyTorch

Créons une classe PyTorch réutilisable qui :
1. Projette l'input `x` vers Q, K, V avec des matrices apprenables
2. Applique la fonction `scaled_dot_product_attention` de l'exercice 5

In [ ]:
class SelfAttention(nn.Module):
    """
    Module de Self-Attention.
    
    Projette l'input x vers Q, K, V puis applique l'attention.
    """

    def __init__(self, embed_dim):
        """
        Args:
            embed_dim: Dimension des embeddings d'entrée
        """
        super().__init__()
        self.embed_dim = embed_dim

        # TODO: Créer 3 couches linéaires pour projeter vers Q, K, V
        # Chaque couche : embed_dim -> embed_dim (utiliser nn.Linear)
        self.W_q = None
        self.W_k = None
        self.W_v = None

    def forward(self, x):
        """
        Args:
            x: Embeddings, shape (batch, seq_len, embed_dim)

        Returns:
            output: Résultat de l'attention
            attention_weights: Poids d'attention
        """
        # TODO: Projeter x vers Q, K, V en utilisant les couches linéaires
        Q = None
        K = None
        V = None

        # TODO: Réutiliser la fonction scaled_dot_product_attention de l'exercice 5
        # (elle fonctionne aussi avec des tenseurs 3D grâce à .transpose(-2, -1))
        output, attention_weights = None, None

        return output, attention_weights

In [ ]:
# Test du module
embed_dim = 32
batch_size = 2
seq_len = 5

attention_layer = SelfAttention(embed_dim)
x = torch.randn(batch_size, seq_len, embed_dim)

output, weights = attention_layer(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")  # Attendu: (2, 5, 32)
print(f"Weights shape: {weights.shape}")  # Attendu: (2, 5, 5)

---

## 7. Visualiser l'attention d'un vrai modèle

Maintenant qu'on a compris et implémenté le mécanisme, regardons ce que ça donne sur un modèle **réellement entraîné**.

On va utiliser **DistilBERT**, une version légère de BERT, pour observer les patterns d'attention appris.

In [ ]:
# Installation de la librairie transformers
!pip install transformers -q

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# Charger un petit modèle pré-entraîné
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_attentions=True)
model.eval()

# Phrase de test (en anglais pour ce modèle)
phrase = "The cat sat on the mat because it was tired"

# Tokenizer la phrase
inputs = tokenizer(phrase, return_tensors="pt")
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# Forward pass (sans calculer les gradients)
with torch.no_grad():
    outputs = model(**inputs)

# Extraire les attentions
attentions = outputs.attentions

print(f"Phrase: {phrase}")
print(f"Tokens: {tokens}")
print(f"Nombre de couches: {len(attentions)}")
print(f"Nombre de têtes par couche: {attentions[0].shape[1]}")

In [ ]:
# Visualiser l'attention d'une tête spécifique
layer = 0   # Première couche (0 à 5)
head = 0    # Première tête (0 à 11)

attention_matrix = attentions[layer][0, head].numpy()

plt.figure(figsize=(10, 8))
plt.imshow(attention_matrix, cmap='Blues')
plt.xticks(range(len(tokens)), tokens, rotation=45, ha='right')
plt.yticks(range(len(tokens)), tokens)
plt.xlabel("Tokens regardés (Keys)")
plt.ylabel("Tokens qui regardent (Queries)")
plt.title(f"Attention réelle - Couche {layer+1}, Tête {head+1}")
plt.colorbar(label="Poids d'attention")

for i in range(len(tokens)):
    for j in range(len(tokens)):
        val = attention_matrix[i, j]
        plt.text(j, i, f'{val:.2f}', ha='center', va='center',
                color='white' if val > 0.3 else 'black', fontsize=7)
plt.tight_layout()
plt.show()

In [ ]:
# Question : Que regarde le pronom "it" ?
# Trouvons son index et regardons sa ligne d'attention

it_index = tokens.index("it")
print(f"'it' est à l'index {it_index}")
print(f"\nAttention de 'it' vers les autres tokens (couche {layer+1}, tête {head+1}):")
print("-" * 40)

for i, (token, weight) in enumerate(zip(tokens, attention_matrix[it_index])):
    bar = "█" * int(weight * 30)
    marker = " ← ?" if token in ["cat", "mat"] else ""
    print(f"  {token:10} {weight:.2f} {bar}{marker}")

**Questions d'analyse** :

1. Le pronom "it" regarde-t-il principalement "cat" ou "mat" ? Pourquoi est-ce logique grammaticalement ?

2. Changez `layer` et `head` dans les cellules ci-dessus. Que remarquez-vous ? (Indice : différentes têtes capturent différentes relations)

3. Essayez d'autres phrases, par exemple :
   - `"The dog chased the cat because it was fast"` (qui est "it" ici ?)
   - `"The trophy didn't fit in the suitcase because it was too big"` (cas ambigu !)

---

## 8. Récapitulatif

### Ce que nous avons appris

1. **L'attention** permet à chaque élément de "regarder" tous les autres
2. **Q, K, V** : Query (ce que je cherche), Key (les étiquettes), Value (le contenu)
3. **Formule** : $\text{softmax}(QK^T / \sqrt{d_k}) \cdot V$
4. **Scaling** : Essentiel pour la stabilité des gradients

### Points clés

| Concept | Rôle |
|---------|------|
| Dot product $QK^T$ | Mesure la similarité |
| Softmax | Transforme en probabilités |
| Scaling $\sqrt{d_k}$ | Stabilise les gradients |
| Self-attention | Q = K = V (chaque mot regarde tous les autres) |

### Prochaine session

Nous verrons le **Multi-Head Attention** : plusieurs "têtes" d'attention qui regardent sous différents angles.

---

## 9. Pour aller plus loin (optionnel)

### Comment entraîne-t-on un Transformer ?

Il existe deux grandes approches selon l'usage du modèle :

### Approche 1 : Prédire le mot suivant (GPT)

Pour les modèles **génératifs** (GPT, LLaMA, etc.), on entraîne le modèle à prédire le prochain mot.

**Objectif** : Entraîner efficacement sur des phrases entières en un seul forward pass.

```
Phrase : "Le chat dort sur"

Sans masque (inefficace) :
  Forward 1 : "Le"           → apprend à prédire "chat"
  Forward 2 : "Le chat"      → apprend à prédire "dort"
  Forward 3 : "Le chat dort" → apprend à prédire "sur"
  → 3 forward passes pour une phrase !

Avec masque causal (efficace) :
  Forward unique : "Le chat dort sur"
    Position 1 (voit "Le")           → apprend à prédire "chat"
    Position 2 (voit "Le chat")      → apprend à prédire "dort"
    Position 3 (voit "Le chat dort") → apprend à prédire "sur"
  → 1 seul forward pass, tout en parallèle !
```

**Le masque causal** permet à chaque position de ne voir que les mots précédents :

```
              Le   chat  dort  sur
      Le    [  ✓     ✗     ✗    ✗  ]
     chat   [  ✓     ✓     ✗    ✗  ]
     dort   [  ✓     ✓     ✓    ✗  ]
      sur   [  ✓     ✓     ✓    ✓  ]
```

**Implémentation** : On met `-∞` aux positions masquées → `softmax(-∞) = 0`

### Approche 2 : Remplir les trous (BERT)

Pour les modèles de **compréhension** (BERT, RoBERTa, etc.) :

```
Entrée :    "Le [MASK] dort sur le [MASK]"
Objectif :   Prédire "chat" et "canapé"
```

Le modèle peut voir tout le contexte (gauche ET droite) pour deviner les mots masqués → pas besoin de masque causal.

### Comparaison

| | GPT (génératif) | BERT (compréhension) |
|--|-----------------|---------------------|
| **Entraînement** | Prédire le mot suivant | Prédire les mots masqués |
| **Contexte** | Passé uniquement | Tout (bidirectionnel) |
| **Masque causal** | ✅ Oui | ❌ Non |
| **Usage** | Génération de texte | Classification, QA, NER |

### Exercice bonus : Implémenter le masque causal

In [ ]:
def scaled_dot_product_attention_with_mask(Q, K, V, mask=None):
    """
    Attention avec masking optionnel.
    
    Args:
        Q, K, V: Query, Key, Value
        mask: Tensor booléen, True = position à masquer
    """
    d_k = K.shape[-1]
    scores = Q @ K.transpose(-2, -1) / math.sqrt(d_k)
    
    # Appliquer le masque (mettre -inf pour les positions masquées)
    if mask is not None:
        scores = scores.masked_fill(mask, float('-inf'))
    
    attention_weights = F.softmax(scores, dim=-1)
    output = attention_weights @ V
    
    return output, attention_weights

# Créer un masque causal (triangulaire)
seq_len = 5
causal_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
print("Masque causal (True = masqué):")
print(causal_mask.int())

In [ ]:
# Test avec masque
Q = torch.randn(seq_len, 8)
K = torch.randn(seq_len, 8)
V = torch.randn(seq_len, 8)

output_masked, weights_masked = scaled_dot_product_attention_with_mask(Q, K, V, causal_mask)

print("Poids d'attention avec masque causal:")
print(weights_masked.round(decimals=2))
print("\nObservation: chaque ligne ne peut voir que les positions précédentes (et elle-même)")